# 在云器Lakehouse的同一张表中进行向量和标量存储

## 方案说明

云器Lakehouse的github_event_issuesevent_embedding表里保存了文本字段，需要对该文本字段进行向量化，并保存在同一张表里的向量字段里，方便进行向量和标量的融合检索。

方案实现了同一张表、同一个VCluster，同时支持了文本数据、向量数据以及倒排索引和向量索引的存储。和传统方式相比，不再需要三套系统（数据仓库、文本检索数据库、向量数据库），最大程度了降低了数据副本数量，避免了数据在三套系统之间的同步。


![image.png](./image/scala_vector_store_in_one_table.png)

- 用到的云器Lakehouse的关键Feature：
  - 向量存储：原生的Vector数据类型，在普通表里直接增加Vector类型的字段；
  - 向量索引：对Vector类型的字段建立索引，加速向量检索的速度；
  - 倒排索引：对文本字段建立倒排索引，加速文本检索的速度；
  - bloom_filter索引：对ID字段建立索引，加速ID过滤；
- 模型服务
  - xinference，本地部署xinference，提供embedding和rerank模型服务；
  - 本方案采用1024维的向量化表示；
- 测试数据集介绍
  - 数据来源是Github的IssuesEvent事件，全文检索字段为其中的issue_body；
  - 向量化表示的是issue_body对应的向量化数据；
  - 全表有1.9亿条记录；


## 数据说明

- 表名：github_event_issuesevent_embedding
- 文本字段：issue_body, string类型
- 向量化字段：issue_body_embedding,vector(float,1024)类型
- 向量化方法：issue_body_embedding的初始值为NULL。调用xinference/ollama本地服务，用bge-m3模型对issue_body字段的文本进行向量化，然后保存在issue_body_embedding字段


## issue_body_embedding字段更新方法
- 单条update方法
  - 符合传统数据库开发者的习惯，数据更实时，达到秒级数据新鲜度高。但是在大数据平台上用SQL进行频繁的update，会带来明显的弊端：
  - 带来大量的小文件需要及时进行合并，以优化性能
  - 需要一直启动VCluster，造成计算成本高
- 批量merge into方法
  - 牺牲了数据新鲜度，从秒级到分钟级
  - 规避了小文件急剧增多的问题
  - 大幅降低了计算成本，每5分钟merge into一次，则计算成本下降幅度达到80%，大幅提升了数据向量化的性价比。这对大数据量的向量化非常重要。

## 文本的向量化表示，xinference
bge-m3为1024维，bge-base-zh-v1.5为768维
本文通过私有部署xinference，提供向量embedding模型服务和rerank召回模型服务 xinference运行在X86 CPU上, xinference的安装和使用请参考这里的[文档](https://inference.readthedocs.io/en/latest/)。

In [1]:
from xinference.client import Client as Xinference_Client  # 添加别名

def get_embedding_xin(
    input_text: str,
    base_url: str = "http://localhost:9998",
    model_name: str = "bge-m3"
) -> list:
    """
    获取文本的嵌入向量
    
    参数:
    input_text (str): 要生成嵌入向量的文本
    base_url (str): Xinference服务器地址，默认为本地服务
    model_name (str): 要使用的模型名称，默认为bge-m3
    
    返回:
    list: 文本的嵌入向量
    """
    # 使用别名创建客户端连接
    client = Xinference_Client(base_url)  # 修改类名调用
    
    # 获取指定模型
    model = client.get_model(model_name)
    embedding = model.create_embedding(input_text)
    # 生成并返回嵌入向量
    return embedding['data'][0]['embedding']

# 使用示例保持不变
if __name__ == "__main__":
    embedding = get_embedding_xin("What is the capital of China?")
    print(f"生成的嵌入向量维度：{len(embedding)}")


生成的嵌入向量维度：1024


## 文本的向量化表示，ollama
文本的向量化表示，1024维embedding函数 ollama运行在Mac M1 ARM上, ollama的安装和使用请参考这里的[文档](https://ollama.com/)。

In [2]:
from ollama import Client

def get_embedding_ollama(text: str, 
                 model: str = 'bge-m3',  # 默认模型
                 host: str = 'http://192.168.6.167:11434') -> list[float]:
    """
    获取文本的向量化表示
    
    参数：
    text (str): 需要向量化的文本内容
    model (str): 使用的embedding模型名称，默认为深度求索的embedding模型
    host (str): Ollama服务器地址，格式为http://IP:PORT
    
    返回：
    list[float]: 文本的向量表示（浮点数列表）
    """
    try:
        client = Client(host=host)
        response = client.embed(
            model=model,
            input=text.strip()  # 去除首尾空白字符
        )
        return response['embeddings'][0]
    except Exception as e:
        print(f"获取embedding失败: {str(e)}")
        return None

# 使用示例
if __name__ == "__main__":
    embedding = get_embedding_ollama(
        text="为什么天空是蓝色的？"
    )
    
    if embedding is not None:
        print(f"向量维度: {len(embedding)}")

向量维度: 1024


## 安装云器Zettapark

In [3]:
# !pip install -U clickzetta-zettapark-python



---



## 通过ZettaPark连接到云器Lakehouse(& without PySpark)

In [4]:
import time
from clickzetta.zettapark.session import Session
import clickzetta.zettapark.functions as f
from clickzetta.zettapark import Session, DataFrame
from clickzetta.zettapark.functions import udf, col
from clickzetta.zettapark.types import IntegerType
import clickzetta.zettapark.types as T

In [5]:
import json
from clickzetta.zettapark.session import Session
import logging
logging.getLogger("clickzetta.zettapark").setLevel(logging.ERROR)

# 从配置文件中读取参数
with open('config-vector.json', 'r') as config_file:
    config = json.load(config_file)

print("正在连接到云器Lakehouse.....\n")

# 创建会话
session = Session.builder.configs(config).create()


print("连接成功！...\n")


正在连接到云器Lakehouse.....

连接成功！...



In [ ]:
## 查看计算资源详细情况
session.sql(f"desc vcluster extended {config['vcluster']}").to_pandas()

## 从云器Lakehouse表中查询待向量化的文本数据

In [9]:
begin_date = "2024-06-01"
end_date = "2024-06-02"

In [10]:
to_be_embedded = session.sql(f"SELECT row_id, partition_date,issue_body from github_event_issuesevent_embedding where partition_date >= '{begin_date}' and partition_date <= '{end_date}' and issue_body_embedding is NULL LIMIT 100000;").collect()

In [11]:
len(to_be_embedded)

49312

## 保存向量数据 
### 方法一：将文本数据向量化后merge into进表里

In [ ]:
# 定义批量大小
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

max_workers = 6

batch_size = 500
batches = []

updatesSchema = T.StructType(
    [
        T.StructField("row_id", T.IntegerType()),
        T.StructField("partition_date", T.StringType()),
        T.StructField("issue_body_embedding", T.StringType())
    ]
)

# 将待处理数据分批
for i in range(0, len(to_be_embedded), batch_size):
    batch = to_be_embedded[i:i + batch_size]
    batches.append(batch)

# 定义一个函数用于处理单条记录的嵌入生成
def process_row(row, use_alternate_method=False):
    """
    处理单条记录，生成对应的嵌入。
    如果 use_alternate_method 为 True，则调用 get_embedding_ollama。
    """
    row_id = row["row_id"]
    issue_body_text = row["issue_body"]
    partition_date = row["partition_date"]

    try:
        # 根据标志调用不同的嵌入生成函数
        if use_alternate_method:
            issue_body_embedding = get_embedding_ollama(issue_body_text)
        else:
            issue_body_embedding = get_embedding_xin(issue_body_text)

        return (row_id, partition_date, issue_body_embedding)
    except Exception as e:
        print(f"Error generating embedding for row {row_id} using {'ollama' if use_alternate_method else 'xin'}: {e}")
        return None
for batch in batches:
    try:
        # 使用 ThreadPoolExecutor 进行并发嵌入生成
        update_rows = []
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for idx, row in enumerate(batch):
                # 每隔一条记录切换到 get_embedding_ollama
                use_alternate_method = (idx % 2 == 1)
                futures.append(executor.submit(process_row, row, use_alternate_method))

            for future in as_completed(futures):
                result = future.result()
                if result is not None:
                    update_rows.append(result)

        # 将生成的结果保存到临时表
        if update_rows:
            update_df = pd.DataFrame(update_rows, columns=["row_id", "partition_date", "issue_body_embedding"])
            try:
                zetta_update_df = session.create_dataframe(update_df, schema=updatesSchema)
                zetta_update_df.write.mode("overwrite").save_as_table("issue_body_embedding_updates")
            except Exception as save_error:
                print(f"Error saving batch to table: {save_error}")
    except Exception as batch_error:
        print(f"Error processing batch: {batch_error}")
        
    issue_body_embedding_updates_len =  session.table("issue_body_embedding_updates").count()
    print(f"issue_body_embedding_updates len = {issue_body_embedding_updates_len}")
    
    merge_query = """
                MERGE INTO github_event_issuesevent_embedding AS target
                USING issue_body_embedding_updates AS source
                ON target.partition_date = source.partition_date
                AND target.row_id = source.row_id
                WHEN MATCHED THEN
                  UPDATE SET issue_body_embedding = cast(source.issue_body_embedding as vector(1024))
                """
    try:
        session.sql(merge_query).collect()
    except Exception as merge_error:
        print(f"Error during MERGE operation: {merge_error}")
        
try:
    session.sql("DROP TABLE IF EXISTS issue_body_embedding_updates").collect()
except Exception as drop_error:
    print(f"Error dropping temporary table: {drop_error}")

issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_updates len = 500
issue_body_embedding_upda

### 方法二：将文本数据向量化后update进表里（不建议使用此方法，原因建方案介绍）

In [ ]:
# 循环处理每一行需要生成嵌入的内容
for row in to_be_embedded:
    try:
        # 获取当前行的元数据
        row_id = row["row_id"]
        issue_body_text = row["issue_body"]
        partition_date = row["partition_date"]
        
        # 调用嵌入生成函数
        issue_body_embedding = get_embedding_xin(issue_body_text)
        
        # 执行SQL更新
        update_stmt = f"""
            USE VCLUSTER DEFAULT;
            UPDATE github_event_issuesevent_embedding
            SET issue_body_embedding = cast({issue_body_embedding} as vector(1024))
            WHERE partition_date = "{partition_date}" and row_id = {row_id}
        """
        session.sql(update_stmt).collect()
        
    except Exception as e:
        print(f"Error processing row {row_id}: {str(e)}")
        # 可以根据需要添加重试逻辑或记录错误


In [ ]:
# session.close()

## 下一步
02在云器Lakehouse的同一张表中进行向量和标量检索